In [85]:
# INSTALL ALL THE REQUIRED PACKAGES.

!pip install -q langchain langchain-openai langchain-community faiss-cpu tiktoken

In [86]:
# IMPORT REQUIRED PACKAGES.

from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

import os

In [87]:
# SET OPENAI API KEY.

os.environ["OPENAI_API_KEY"] = ""

In [88]:
loader = TextLoader("/content/catan-rules.txt")
loader

In [89]:
documents = loader.load()
documents

[Document(page_content="The Settlers of Catan - or as it’s now officially called, just Catan - is a modern classic when it comes to board games. This means there's a good chance you already know how to play Catan - but there's never a bad time to brush up on the official Catan rules to make sure you're playing correctly. Here you'll be able to do exactly that.\nKlaus Teuber’s game of trading and expansion is considered one of the most influential and important board games of all time, sparking a golden age of modern tabletop gaming and shifting more than 20 million copies in the 30-plus years since it was first released. It seems everyone from Frozen star Kristen Bell to pop singer Carly Rae Jepsen is a fan.\nWhile there have been plenty of expansions and spin-offs released for Catan over the years - there’s even a big-screen movie in the works - there are still plenty of people who have never played this classic game.\nIf you’re yet to experience the legendary board game or simply nee

In [90]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len
)

In [91]:
docs = text_splitter.split_documents(documents)
docs[0]

Document(page_content="The Settlers of Catan - or as it’s now officially called, just Catan - is a modern classic when it comes to board games. This means there's a good chance you already know how to play Catan - but there's never a bad time to brush up on the official Catan rules to make sure you're playing correctly. Here you'll be able to do exactly that.", metadata={'source': '/content/catan-rules.txt'})

In [92]:
len(docs)

29

In [93]:
embedding = OpenAIEmbeddings()

In [94]:
vec_library = FAISS.from_documents(docs, embedding=embedding)

## **Similarity Search**

In [95]:
Query1 = "What resources does the players get initially?"

In [96]:
Query1_answer = vec_library.similarity_search(Query1)

In [97]:
Query1_answer[0]

Document(page_content='For the recommended beginner setup, you take the resources of the land surrounding the settlement marked by the white star in the rulebook: Yellow receives a sheep, a wood and a wheat. Red gets a wheat, a wood and a sheep. Blue gets a brick, a sheep and an ore. White gets an ore, a wood and a sheep.\nWhen using the experienced player setup, you instead receive resources from each of the tiles next to your second settlement.\nCatan rules\nIn Catan, each player turn is split into three phases:', metadata={'source': '/content/catan-rules.txt'})

In [98]:
Query1_answer[1]

Document(page_content='Each player should be given a building cost card - this will remind you what resources you’ll need to build further settlements, roads and cities, as well as buying development cards.\nThe Longest Road and Largest Army cards, as well as the two dice, should be placed alongside the game board.\nSeparate the resources into piles of their type: sheep, wood, brick, ore and wheat.', metadata={'source': '/content/catan-rules.txt'})

In [99]:
docs_and_scores = vec_library.similarity_search_with_score(Query1)

In [100]:
docs_and_scores[0]

(Document(page_content='For the recommended beginner setup, you take the resources of the land surrounding the settlement marked by the white star in the rulebook: Yellow receives a sheep, a wood and a wheat. Red gets a wheat, a wood and a sheep. Blue gets a brick, a sheep and an ore. White gets an ore, a wood and a sheep.\nWhen using the experienced player setup, you instead receive resources from each of the tiles next to your second settlement.\nCatan rules\nIn Catan, each player turn is split into three phases:', metadata={'source': '/content/catan-rules.txt'}),
 0.33414954)

## **Retriever with OpenAI**

In [101]:
retriever = vec_library.as_retriever()

In [102]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [103]:
Query2 = "What happens when a player plays a seven?"

In [104]:
results = qa.invoke(Query2)

In [105]:
results

{'query': 'What happens when a player plays a seven?',
 'result': '\nWhen a player plays a seven in Catan, they activate the robber. This means that any player with more than seven resource cards in their hand must return half of them to the resource stacks. The player who played the seven also gets to move the robber to a new location on the board, covering the number token and preventing players from gaining resources from adjacent settlements. This lasts until the robber is moved again. Additionally, playing a seven also counts as playing a knight, which can earn the player a two-point bonus for having the Largest Army.'}

## **Save FAISS index**

In [106]:
vec_library.save_local("catan_vector_index")